# C# pattern matching by type

The 2019 MSDN article, “[Pattern Matching in C# 8.0](https://docs.microsoft.com/en-us/archive/msdn-magazine/2019/may/csharp-8-0-pattern-matching-in-csharp-8-0),” by Filip Ekberg will be used as a guide around ‘modern’ pattern-matching in C#.

The real-world example (from my day-job life) that brings pattern matching in my life is the need to map a class to its corresponding ‘handler’ class:

In [ ]:
public interface IThingHandler<TThing>
{
    void DoTheThing(TThing input) =>
        Console.WriteLine($"You have done the thing with {typeof(TThing).Name}.");
}

public class ClassOne {}
public class ClassOneHandler : IThingHandler<ClassOne> {}

public class ClassTwo {}
public class ClassTwoHandler : IThingHandler<ClassTwo> {}

public class ClassThree {}
public class ClassThreeHandler : IThingHandler<ClassThree> {}


The `IThingHandler<TThing>` is a generic interface with a [default implementation](https://docs.microsoft.com/en-us/dotnet/csharp/language-reference/proposals/csharp-8.0/default-interface-methods) that can be assigned to a `*Handler` class in `DoTheThingWithTheThing<TThing>`:

In [ ]:
public void DoTheThingWithTheThing<TThing>(TThing data)
{
    switch (data)
    {
        case ClassOne thing:
            {
                IThingHandler<ClassOne> handler = new ClassOneHandler();
                handler.DoTheThing(thing);
            }
            break;

        case ClassTwo thing:
            {
                IThingHandler<ClassTwo> handler = new ClassTwoHandler();
                handler.DoTheThing(thing);
            }
            break;

        case ClassThree thing:
            {
                IThingHandler<ClassThree> handler = new ClassThreeHandler();
                handler.DoTheThing(thing);
            }
            break;

        default:
            var type = typeof(TThing);
            throw new NotSupportedException($"{nameof(Type)} {type.Name} is not supported.");
    }
}

`DoTheThingWithTheThing<TThing>` has a `switch` statement written in a form that Filip Ekberg calls, “a lightweight version of pattern matching” which allows imperative code blocks and functional-style pattern matching. This is a C# 7.0 form of pattern matching that is easy to understand by more traditional C# developers—and it works just fine:

In [ ]:
var inputTwo = new ClassTwo();

DoTheThingWithTheThing<ClassTwo>(inputTwo)

You have done the thing with ClassTwo.


## C# 8.0 pattern-matching syntax

Let me write the above in what I think is C# 8.0 syntax. I can see that this does not compile:

```csharp
public void DoTheThingWithTheThing2019<TThing>(TThing data)
{
    data switch {
        ClassOne thing =>
            {
                var handler = new ClassOneHandler();
                handler.DoTheThing(thing);
            },

        ClassTwo thing =>
            {
                var handler = new ClassTwoHandler();
                handler.DoTheThing(thing);
            },

        ClassThree =>
            {
                var handler = new ClassThreeHandler();
                handler.DoTheThing(thing);
            },

        _ =>
            {
                var type = typeof(TThing);
                throw new NotSupportedException($"{nameof(Type)} {type.Name} is not supported.");
            }
    };
}
```

My experience with F# is telling me that those curly braces will _not_ be supported in the C# 8.0 design because, like what we have in F#, modern pattern-matching in C# is designed for _expressions_:

In [ ]:
public void DoTheThingWithTheThing2019<TThing>(TThing data)
{
    IThingHandler<TThing> handler = data switch
    {
        ClassOne _ => (IThingHandler<TThing>)new ClassOneHandler(),

        ClassTwo _ => (IThingHandler<TThing>)new ClassTwoHandler(),

        ClassThree _ => (IThingHandler<TThing>)new ClassThreeHandler(),

        _ => throw new NotSupportedException($"{nameof(Type)} {typeof(TThing).Name} is not supported.")
    };

    handler.DoTheThing(data);
}

I am more uncomfortable with the casting operations (`(IThingHandler<TThing>)`) than the pattern matching syntax. I would like to be wrong about needing these generics-based operations. Nevertheless, this more succinct 2019 form works just like the “lightweight” form:

In [ ]:
var inputThree = new ClassThree();

DoTheThingWithTheThing2019<ClassThree>(inputThree)

You have done the thing with ClassThree.


[Bryan Wilhite is on LinkedIn](https://www.linkedin.com/in/wilhite)🇺🇸💼
